In [6]:
from bs4 import BeautifulSoup
import urllib.request
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from geopy.geocoders import Nominatim
import json 
import time


opentable_url = 'https://www.opentable.com/raleigh-durham-chapel-hill-restaurant-listings'
chrome_driver_path = '/Users/orgil/Downloads/chromedriver'


In [77]:
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(chrome_driver_path,options=options)
driver.get(opentable_url)


In [78]:
def get_menu_items():
    restaurant = {}
    categories = driver.find_elements_by_class_name("button-menu-link__1j5RFIKk")
    try:
        full_menu_btn = driver.find_element_by_class_name("button-static__33d7U9QU")
        full_menu_btn.click()
    except NoSuchElementException:
        return
    rest_name = driver.find_element_by_class_name("a6481dc2").text
    address = driver.find_element_by_class_name("_3ddfcf5c").text
    geolocator = Nominatim(user_agent="open_table")
    location = geolocator.geocode(address,timeout=10)
    restaurant['Name'] = rest_name
    try:
        restaurant['Location'] = [location.latitude, location.longitude]
    except AttributeError:
        restaurant['Location'] = address

   
    menu_items = []
    for i in range(len(categories)):
        WebDriverWait(driver,3)
        categories[i].click()
        WebDriverWait(driver,3)
        items = driver.find_elements_by_class_name("menu-item__2ZxJOnTY")
        print(len(items))
        for item in items:
            parts = item.text.split('\n')
            menu_items.append(get_menu_item(parts))
    restaurant['Menu'] = menu_items
    print(restaurant)
    return restaurant


In [79]:
def get_menu_item(parts):
    dish = {}
    if len(parts) >= 2:
        dish['name'] = parts[0]
        dish['price'] = parts[1]
        if len(parts) > 2:
            dish['desc'] = parts[2]
    return dish


In [80]:
def scrape_page():
    
    WebDriverWait(driver,10).until(EC.invisibility_of_element((By.XPATH, "//*[@id='loading_animation']")))

    cur_win = driver.current_window_handle # get current/main window
    page = driver.find_elements_by_class_name("rest-row-name") # getting all restaurant links in a page
    
    rests_in_page = []
    
    for link in page:
        try:
            WebDriverWait(driver,10).until(EC.visibility_of(link))
            link.click()
        except ElementClickInterceptedException:
            link.click()


        driver.switch_to.window([win for win in driver.window_handles if win !=cur_win][0]) # switch to new window

        print(driver.current_url)
        rests_in_page.append(get_menu_items())

        driver.close() # close new window
     
        driver.switch_to_window(cur_win) # switch back to main window

    return rests_in_page



In [81]:

count = 1
while True:
    # wait until loading animation finishes
    WebDriverWait(driver,10).until(EC.invisibility_of_element((By.XPATH, "//*[@id='loading_animation']")))
    restaurants = scrape_page()
    
    with open("opentable_page"+str(count)+".json", "w") as write_file:
        json.dump(restaurants, write_file)
        
    # make sure next page button is clickable
    WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'pagination-next')))
    next_page_btn = driver.find_element_by_class_name('pagination-next')
    
    next_page_btn.click()
    count += 1




https://www.opentable.com/r/seasons-52-raleigh?avt=eyJ2IjoxLCJtIjoxLCJwIjoxfQ&corrid=74c81e0c-3b88-4fbd-aee8-66e98e2fa445
53
23
20
102
{'Name': 'Seasons 52 - Raleigh', 'Location': [1.2887091, 103.805221834954], 'Menu': [{'name': 'Pesto Chicken and Fresh Mozzarella', 'price': 'balsamic Cipollini onions, wood-grilled red peppers, arugula (460 cal)'}, {'name': 'Four-Mushroom', 'price': 'goat cheese, truffle oil, scallions (430 cal)'}, {'name': 'Broccoli, Bacon and Gruyère', 'price': 'sriracha cheese sauce (580 cal)'}, {'name': 'Roasted Tomato', 'price': 'mozzarella, Parmesan, basil (420 cal)'}, {'name': 'All-Natural Pepperoni', 'price': 'roasted tomato, mozzarella, Parmesan, basil (570 cal)'}, {'name': 'Lobster and Fresh Mozzarella', 'price': 'wood-grilled red peppers, scallions, lobster sour cream (420 cal)'}, {'name': 'Lump Crab Cake', 'price': 'Maryland-style, mustard sauce (340 cal)'}, {'name': 'Grilled Shrimp and Avocado Cocktail', 'price': 'avocado-lime crema (210 cal)'}, {'name': '

/Users/orgil/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: use driver.switch_to.window instead


https://www.opentable.com/r/crawford-and-son-raleigh?avt=eyJ2IjoxLCJtIjoxLCJwIjowfQ&corrid=74c81e0c-3b88-4fbd-aee8-66e98e2fa445
20
52
{'Name': 'Crawford & Son', 'Location': 'http://www.crawfordandsonrestaurant.com/', 'Menu': [{'name': 'Chilled Snap Pea Soup', 'price': '$11.00', 'desc': 'Saffron Pickled Shrimp, Cucumber, Radish'}, {'name': 'Baby Lettuces', 'price': '$10.00', 'desc': 'Sherry Vinaigrette, Smoked Pecans, Manchego'}, {'name': 'Asparagus Salad', 'price': '$13.00', 'desc': 'Salted Strawberries, Olive Oil, Walnuts, Feta'}, {'name': 'Buttered Blue Crab', 'price': '$15.00', 'desc': 'Parisienne Gnocchi, Peas, Leeks, Parmesan'}, {'name': 'Goat Bolognese', 'price': '$14.00', 'desc': 'Pappardelle Pasta, Soft Egg, Aged Caprotto'}, {'name': 'Shaved Kohlrabi', 'price': '$12.00', 'desc': 'Fermented Garlic, Sesame, Thai Basil'}, {'name': 'Scallop Tartare*', 'price': '$14.00', 'desc': 'Yuzu Kosho, Crispy Taro, Buttermilk Ice'}, {'name': 'Beef Carpaccio*', 'price': '$15.00', 'desc': 'Tapen

https://www.opentable.com/reys?avt=eyJ2IjoxLCJtIjoxLCJwIjoxfQ&corrid=74c81e0c-3b88-4fbd-aee8-66e98e2fa445
69
{'Name': "Rey's", 'Location': [1.2887091, 103.805221834954], 'Menu': [{'name': 'Shrimp Orleans', 'price': '$13.00', 'desc': 'Four gulf shrimp sautéed in a New Orleans butter'}, {'name': 'Shrimp Cocktail', 'price': '$13.00', 'desc': 'Jumbo white shrimp served with cocktail and spicy remoulade sauce'}, {'name': 'Crabmeat Cocktail', 'price': '$16.00', 'desc': 'Colossal crabmeat served with cocktail and spicy remoulade sauce'}, {'name': 'Pan-Seared Scallops*', 'price': '$14.00', 'desc': 'With a garlic-lime butter'}, {'name': 'Fried Calamari', 'price': '$12.00', 'desc': 'With a basil aioli sauce'}, {'name': 'Seared Ahi Tuna*', 'price': '$12.00', 'desc': 'With wasabi cream and pickled ginger'}, {'name': 'Grilled Brie Served with Fresh Fruit and Toast Points', 'price': '$11.00'}, {'name': 'Fried Oysters', 'price': '$12.00', 'desc': 'Served over a New Orleans style succotash topped with

https://www.opentable.com/r/mandolin-raleigh?avt=eyJ2IjoxLCJtIjoxLCJwIjowfQ&corrid=74c81e0c-3b88-4fbd-aee8-66e98e2fa445
20
25
10
8
19
{'Name': 'Mandolin', 'Location': [54.22559135, -1.17746331727674], 'Menu': [{'name': 'Charcuterie Plate', 'price': '$16.00', 'desc': 'Assorted Cured Meats, Sour Dough Croûtes, Whole Grain Mustard, Pickled Celery, Cornichons'}, {'name': 'Chadwick Creek Oysters on the Half Shell', 'price': 'House Cocktail Sauce, Hot Sauce', 'desc': 'each'}, {'name': 'Panzanella Salad', 'price': '$10.50', 'desc': 'Country Croutons, Smoked Farmers Cheese, Beets, Caper Berries, Roasted Tomatoes, House-Made Pickles, Herbs'}, {'name': 'Heirloom Tomato Salad', 'price': '$11.00', 'desc': 'Cucamelons, Castelvetrano Olives, Providence Cheese, Sherry Vinaigrette, Garden Herbs'}, {'name': 'North Carolina Butter Beans', 'price': '$9.50', 'desc': 'Espelette Pepper and House Made Bottarga'}, {'name': 'Tuna Tartare', 'price': '$12.50', 'desc': 'Watermelon, Cucumber, Coriander-Avocado Yog

...

https://www.opentable.com/r/flights-raleigh?corrid=74c81e0c-3b88-4fbd-aee8-66e98e2fa445
https://www.opentable.com/r/taste-raleigh?corrid=74c81e0c-3b88-4fbd-aee8-66e98e2fa445
https://www.opentable.com/r/pullen-place-cafe-and-catering-raleigh?corrid=74c81e0c-3b88-4fbd-aee8-66e98e2fa445
https://www.opentable.com/r/hibachi-japan-raleigh?corrid=74c81e0c-3b88-4fbd-aee8-66e98e2fa445
https://www.opentable.com/r/boulted-bread-raleigh?corrid=74c81e0c-3b88-4fbd-aee8-66e98e2fa445
https://www.opentable.com/r/black-cat-cafe-raleigh?corrid=74c81e0c-3b88-4fbd-aee8-66e98e2fa445
5
{'Name': 'Black Cat Cafe', 'Location': 'Claim this listing', 'Menu': [{'name': '2 Eggs Any Style', 'price': '$7.25', 'desc': 'Served with choice of breakfast meat, grits or seasonal fruit and toast.'}, {'name': 'Bruschetta', 'price': '$6.95', 'desc': 'Toasted, house-made focacia bread topped with creamy mascarpone cheese,roasted red peppers and basil pesto, topped with grated parmesan.'}, {'name': 'Thai Curry Chicken Salad', '

https://www.opentable.com/r/hangover-grill-raleigh?corrid=74c81e0c-3b88-4fbd-aee8-66e98e2fa445
https://www.opentable.com/r/east-village-grill-and-bar-raleigh?corrid=74c81e0c-3b88-4fbd-aee8-66e98e2fa445
37
80
{'Name': 'East Village Grill & Bar', 'Location': 'Claim this listing', 'Menu': [{'name': 'Onion Rings', 'price': '$4.25', 'desc': 'served with ranch'}, {'name': 'Pickle Chips', 'price': '$4.25', 'desc': 'our famous hand battered pickle chips served with house made ranch dressing'}, {'name': 'Cheese Fries', 'price': '$6.00', 'desc': 'melted cheese, bacon, sour cream, scallions'}, {'name': 'Village Chips & ranch', 'price': '$3.50', 'desc': 'thinly cut potato slices served with our house made ranch dressing'}, {'name': 'Nacho Supreme', 'price': '$8.50', 'desc': 'sour cream, salsa, queso, lettuce, tomato, fresh jalapenos'}, {'name': 'Veggies & Hummus', 'price': '$5.00', 'desc': 'House made hummus served with fresh veggies'}, {'name': 'Chicken Fingers', 'price': '$7.25', 'desc': '6 fres

https://www.opentable.com/r/buffalo-brothers-pizza-and-wing-company-raleigh?corrid=74c81e0c-3b88-4fbd-aee8-66e98e2fa445
28
{'Name': 'Buffalo Brothers Pizza And Wing Company', 'Location': 'Claim this listing', 'Menu': [{'name': 'Fried Pickles', 'price': '$6.99', 'desc': 'fresh made to order crinkle cut dill chips lightly battered and deep fried! really, who doesn’t love a fried pickle? served with a side of blue cheese or ranch'}, {'name': 'Philly Cheesesteak Egg Rolls', 'price': '$6.99', 'desc': 'finely diced peppers, onions and mushrooms wrapped up with shaved ribeye and gooey cheese in an crispy fried egg roll'}, {'name': 'Spinach Artichoke Dip', 'price': '$7.99', 'desc': 'our homemade creamy blend of spinach and cheeses baked hot and served with fresh tortilla chips'}, {'name': 'Fried Mozzarella', 'price': '$7.49', 'desc': 'a block of creamy whole milk mozzarella, breaded & fried served in warm marinara sauce. made fresh daily - you’ll never want frozen “stix” again'}, {'name': 'Che

https://www.opentable.com/r/dalat-oriental-restaurant-raleigh?corrid=74c81e0c-3b88-4fbd-aee8-66e98e2fa445
10
{'Name': 'Dalat Oriental Restaurant', 'Location': 'Claim this listing', 'Menu': [{'name': '2 Spring Roll', 'price': 'Carrot, vermicelli and cabbage inside a crispy shell. served with a house fish sauce and fresh lettuce', 'desc': 'Veggie'}, {'name': '2 Summer Roll', 'price': 'Vermicelli, choice of filling, and fresh vegetables rolled into a clear and soft rice shell. Served with a peanut sauce', 'desc': 'Veggie'}, {'name': 'Egg Drop Soup', 'price': '$2.99', 'desc': 'Old fashioned, thick eggy soup'}, {'name': 'Hot and Sour Soup', 'price': '$2.99', 'desc': 'Thick and hearty spicy soup with pork, eggs, bamboo shoots and mushrooms'}, {'name': 'Beef Noodle Soup', 'price': '$11.99', 'desc': 'Pho. Rice noodles with rare sliced steak, fatty brisket, and soft tendon. Served with fresh bean sprouts, lime and lemon, jalapeno peppers, cilantro and basil leaves'}, {'name': 'Chicken Noodle So

IndexError: list index out of range